In [10]:
# We have gathered 285 news articles from sources including BBC News, Fox News, and BuzzFeed for testing purposes.

In [1]:
#Importing neccessary libraries
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
#Create a function to instantiate the Safari webdriver when a certain url is provided
def instantiate(url):
    driver = webdriver.Safari()
    driver.get(url)
    return driver

In [3]:
df = []

#Create a function that takes the instantiated driver and the base_xpath and the label to be extracted
def scrape_headlines(driver,base_xpath,label):
    # Find the HTML elements containing the news headlines
    headlines = driver.find_elements(By.XPATH, base_xpath)  # Adjust class or tag accordingly

    # Extract and print the headlines
    for headline in headlines:
        data = {}
        data['headline'] = headline.text
        data['label'] = label
        df.append(data)

In [4]:
#Helper function to automate scrolling to the end of the page
def scroll_to_end():
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
        time.sleep(2)  # Adjust the sleep time based on your needs

In [5]:
# Extract news articles from fox news, the label is 0 - Non Clickbait
url = "https://www.foxnews.com"
base_xpaths = ["//*[@id='wrapper']/div[2]/*/main/*/div/*/div[2]/header/h3/a",
"//*[@id='wrapper']/div[2]/*/main/div[2]/*/div[2]/header/h3/a"]
driver = instantiate(url)

for base_xpath in base_xpaths:
    scrape_headlines(driver,base_xpath,0)
    
# Close the browser window
driver.quit()

In [6]:
# Extract news articles from bbc news, the label is 0 - Non Clickbait

url = "https://www.bbc.com/news/us-canada"
base_xpath = "//*[@id='__next']/main/section[5]/div/div[2]/div[1]/*/div/a/div/div[2]/div[2]/div[1]/div/h2"
driver = instantiate(url)
scrape_headlines(driver,base_xpath,0)

button_xpath = "//*[@id='__next']/main/section[5]/div/div[2]/div[2]/div/button[9]"

for i in range(0,4):
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, button_xpath)))

    next_button.click()
    time.sleep(3)
    scrape_headlines(driver,base_xpath,0)

driver.quit()

In [7]:
# Extract news articles from buzzfeed , the label is 1 - Clickbait

url = "https://www.buzzfeednews.com"
base_xpath = "//*[@id='mod-main-feed-1']/*/div[1]/*/span[2]/p"
driver = instantiate(url)

for i in range(1,5):
    scroll_to_end()
    button_xpath = f"//*[@id='mod-show-more-{i}']/div/button"
    more_stories_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, button_xpath)))
    time.sleep(3)
    more_stories_button.click()
    time.sleep(3)

scrape_headlines(driver,base_xpath,1)

# Close the browser window
driver.quit()

In [9]:
#Convert the data collected to a dataframe and save is as a .csv file
data = pd.DataFrame(df)
data.to_csv('scrapped_data.csv', index=False)
print(data)

                                              headline  label
0    Dem-appointed Colorado justice warns of reperc...      0
1    WATCH LIVE: VP Harris leads meeting ahead of N...      0
2    DOJ announces indictment in connection with 19...      0
3    Biden says 'no question' Trump supported insur...      0
4    Brittany Mahomes fires back at 'rude a-- peopl...      0
..                                                 ...    ...
280  Joshua was spotted out with Lupita just a day ...      1
281  “He calls, and we speak on the phone one last ...      1
282  Taylor was seen literally shaking with fear wh...      1
283                                It’s all adding up…      1
284  Almost a decade after Britney and Justin’s mes...      1

[285 rows x 2 columns]
